<a href="https://colab.research.google.com/github/RodolfoFerro/satelitesyneuronas/blob/main/notebooks/Procesamiento_de_im%C3%A1genes_e_im%C3%A1genes_de_galaxias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de imágenes e imágenes de galaxias

**¡Te doy la más cordial bienvenida! 👋**

En ese cuaderno, a una imagen con 2 dimensiones de color, le agregamos una tercera dimensión con pixeles negros. Ahora, a una imagen con 3 dimensiones de color _–RGB–_ le vamos a agregar 2 dimensiones extra con pixeles rojo y azul. ¡Esto nos va a dar información de dónde hay luz UV e IR a partir de información captada por un satélite!

**Spoiler:** Para ello, usaremos una imagen de una galaxia a lo largo de este ejercicio.

## Importar módulos

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio.v2 as imageio
import numpy as np
import pylab
import os

## Definir funciones

Esta función genera una película en formato .gif a partir de una serie de imágenes en formato .png.

In [ ]:
def makemovie(n, movie_name='movie.gif'):
    """Genera una animación GIF."""

    images = []
    filenames = []

    for i in range(n):
        filenames.append("frame" + str(i) + ".png")

    for filename in filenames:
        images.append(imageio.imread(filename))

    imageio.mimsave(movie_name, images)

Esta función mapea los pixeles de una imagen al intervalo [0,1] usando una transformación lineal.  

In [ ]:
def linear_map(x):
    """Mapea data linealmente al intervalo [0,1]."""

    m = np.min(x)
    M = np.max(x)

    return (x - m) / (M - m)

Esta funcion te permite visualizar imagenes de manera mas "bonita".

In [ ]:
def visualizar_imagen(imagen):
    """Función para visualizar una imagen."""

    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111)
    ax.imshow(imagen, interpolation="gaussian")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

Esta función te permite obtener el "histograma" de los valores de los pixeles de una imagen.

In [ ]:
def histograma_imagen(imagen):
    """Función para obtener el histograma de una imagen."""

    plt.style.use("seaborn-v0_8")
    tamano = 1
    for s in imagen.shape:
        tamano *= s

    img_vec = np.reshape(imagen, tamano)

    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(111)
    hist = plt.hist(img_vec, 50)

    plt.xlabel("Valores de pixeles", fontsize=15)
    plt.ylabel("Frecuencia", fontsize=10)

## Lectura de datos de galaxias

Lee los archivos con datos en los 5 canales (RGB y Ultraviolete e Infrarojo) de la siguiente manera.

Estos datos fueron descargados de este sitio:
http://skyview.gsfc.nasa.gov/current/cgi/query.pl

Es importante que descargues y subas a este cuaderno la data correspondiente a una galaxia. Puedes encontrar la data en este link: https://github.com/RodolfoFerro/satelitesyneuronas/tree/main/assets/data

**O también, buscamos a la galaxia M51, y tomamos los datos SDSSdr7 `[g,i,r,u,z]`.**

In [ ]:
# Los archivos que corresponden a R, G y B se llaman 'i', 'r' y 'g'.
R_fits = fits.open('i.fits')
G_fits = fits.open('r.fits')
B_fits = fits.open('g.fits')


# Los archivos .fits con datos UV e IR se llaman u.fits y z.fits.
# Haz el equivalente con ellos que lo que hicimos con los tres
# archivos de arriba. Asignalos a UV_fits y IR_fits.
########


########

In [ ]:
# Código de utilería para cargar la data automáticamente.
!wget https://github.com/RodolfoFerro/satelitesyneuronas/raw/main/assets/data/g.fits
!wget https://github.com/RodolfoFerro/satelitesyneuronas/raw/main/assets/data/i.fits
!wget https://github.com/RodolfoFerro/satelitesyneuronas/raw/main/assets/data/r.fits
!wget https://github.com/RodolfoFerro/satelitesyneuronas/raw/main/assets/data/u.fits
!wget https://github.com/RodolfoFerro/satelitesyneuronas/raw/main/assets/data/z.fits

Los astrónomos trabajan con una archivos que se llaman formato "FITS". Adentro de estos están los datos crudos de las imágenes. En las siguientes lineas, tomamos las imágenes R, G, B, UV y IR

In [ ]:
img_R = R_fits['PRIMARY'].data
img_G = G_fits['PRIMARY'].data
img_B = B_fits['PRIMARY'].data

# Ahora tú abre las imagenes img_UV y imv_IR.
########


########

# El RGB de la galaxia

## Combinemos R, G y B en una sola imagen

Primero, averiguamos el ancho y alto de la imagen.

In [ ]:
ancho_imagen = img_R.shape[0]
altura_imagen = img_R.shape[1]

print("Ancho:", ancho_imagen, "px")
print("Alto:", altura_imagen, "px")

In [ ]:
# Ahora, generamos una matriz de 3 dimensiones llena de puros ceros.
img_RGB = np.zeros((ancho_imagen, altura_imagen, 3))

# Ahora, ponemos img_R (el rojo) la posición en img[:, :, 0]
# y luego img_G (el verde) en img[:, :, 1].
img_RGB[:, :, 0] = img_R
img_RGB[:, :, 1] = img_G

# Ahora, tú pon img_B (el azul) la posición en img_RGB[:, :, 2].
########

########

## ¿Cómo se ve la imagen de la galaxia ahorita?

In [ ]:
# Usa la funcion visualizar_imagen para visualizar a img_RGB.
################

################

¿Por qué se ve tan rara? Vamos a ver qué valores tienen los pixeles:

In [ ]:
# Usa la función histograma_imagen para ver el
# histograma de valores de pixeles.
##############

###############

Notemos que que los pixeles no van del 0 al 1, sino que tienen valores mucho más grandes (1000, 2000, etc.)

Una forma común en Astronomía (y muchas otras áreas de la ciencia) de transformar los valores para que vayan del 0 al 1 es la siguiente: sacar el logaritmo base 10 y luego hacer un "mapeo lineal" de 0 a 1.

In [ ]:
# 1. Obtenemos el logaritmo base 10 de los pixeles:
img_RGB_log10 = np.log10(img_RGB)

# 2. Llamamos a la función "linear map":
img_RGB_01 = linear_map(img_RGB_log10)

Verificamos los nuevos valores viendo el nuevo histograma:

In [ ]:
# Grafica el histograma de la nueva imagen, normalizada.
################

################

**¡Ahora sí van del 0 al 1 los valores de los pixeles!**

## ¿Cómo se ve la imagen nueva?

In [ ]:
# Visualiza la nueva imagen.
##################

##################

La imagen arriba tiene 3 canales - RGB...

¡Pero lo emocionante es que tenemos información sobre 2 "dimensiones de color" adicionales - el ultravioleta (UV) y el infrarojo (IR) (los datos almacenados en las variables `img_UV` e `img_IR`)!

Entonces, de manera similar a como le hicimos con el ejercicio para daltonismo, queremos usar *oscilaciones en el tiempo* de pixeles para codificar las 2 dimensiones extras de color.

## Transformación de la imagen

Primero, vamos a transformar de la misma forma que hicimos arriba, los datos de UV e IR a un rango que vaya del 0 al 1.

In [ ]:
# Transforma los datos Ultravioleta (img_UV) e Infrarojo (img_IR)
# al rango [0,1]. Primero saca  el logaritmo base 10 de la señal,
# y luego aplica el mapa lineal (lineal_map).
# Asigna las nuevas imagenes a las variables "img_UV_01" y "img_IR_01".

#################


#################

Veamos el histograma de valores de estos dos canales. Esto nos va a ayudar a decidir qué umbral de intensidad utilizar para seleccionar los pixeles que oscilen en el tiempo.

In [ ]:
# Muestra el histograma de valores de pixeles del
# canal UV (transformado al rango 0-1).
#############

#############

In [ ]:
# Muestra el histograma de valores de pixeles del
# canal IR (transformado al rango 0-1)
#######

#######

**Pregunta:** Con base en los histogramas de UV e IR, ¿qué valores escogerías para los umbrales de cada uno de estos dos canales?

Recuerda que todos los pixeles que tengan un valor por arriba del umbral van a oscilar en el tiempo, mostrando donde hay regiones con alta señal en el canal UV o IR, respectivamente.

In [ ]:
# Escoge el valor para el umbral del canal UV.
# Puedes cambiarlo en base a lo que observes
# en el histograma.
umbral_UV = 0.1

# Escoge el valor para el umbral del canal IR.
umbral_IR = 0.1

## Primera prueba

Hagamos una primera prueba con pixeles en el canal UV que estén por arriba del `umbral_UV`.

**Condición –** Si un pixel en el canal UV está por arriba del `umbral_UV`, y un número aleatorio entre 0 y 1 es menor a 0.2, pinta ese pixel de color azul.

In [ ]:
# Haz una copia de img_RGB_01, llámala img_prueba_UV.
#############


#############

In [ ]:
for i in range(ancho_imagen):  # Este es un doble for loop que itera sobre...
    for j in range(altura_imagen): # ...los renglones y columnas de la matriz
        # Utiliza un "if statement" para que cada vez que
        # se satisfaga la condicion "Si un pixel en el canal UV
        # está por arriba del umbral_UV, y un número aleatorio
        # entre 0 y 1 es menor a 0.2, pinta ese pixel de color azul.
        ################################



        #################################

**¿Cómo se ve esta imagen de prueba?**

In [ ]:
# Visualiza la nueva imagen, img_prueba_UV
##################

##################

**Pregunta:** ¿Qué pasa si cambias el valor del `umbral_UV`? inténtalo y vuelve a generar la imagen de prueba.

Algunos valores sugeridos: `umbral_UV = 0.05` o `0.15`, etc.

Ahora, hagamos ahora una primera prueba con pixeles en el canal IR que estén por arriba del `umbral_IR`.

**Condición –** Si un pixel en el canal IR está por arriba del `umbral_IR`, y un número aleatorio entre 0 y 1 es menor a 0.2, pinta ese pixel de color rojo.

In [ ]:
# Haz lo mismo que hicimos arriba, ahora para el canal IR.
###############

###############

In [ ]:
# Haz lo mismo que hicimos arriba, ahora para el canal IR.
#################



################

In [ ]:
visualizar_imagen(img_prueba_IR)

**Pregunta:** ¿Qué pasa si cambias el valor del `umbral_IR`? inténtalo y vuelve a generar la imagen de prueba.

Algunos valores sugeridos: `umbral_IR = 0.05` o `0.15`, etc.

También puedes intentar cambiar el umbral del número aleatorio, para cambiar la densidad de los pixeles.

## Ahora ambos canales

Ahora, hagamos una prueba donde los pixeles que esten arriba de los umbrales _de los dos canales_ sean de color azul o rojo respectivamente.

Junta las dos condiciones de arriba:

In [ ]:
img_prueba_IR_UV = img_RGB_01.copy()

In [ ]:
for i in range(ancho_imagen):
    for j in range(altura_imagen):
        if img_IR_01[i,j] > umbral_IR and np.random.random() < 0.2:
            img_prueba_IR_UV[i,j,0] = 1
            img_prueba_IR_UV[i,j,1] = 0
            img_prueba_IR_UV[i,j,2] = 0
        if img_UV_01[i,j] > umbral_UV and np.random.random() < 0.2:
            img_prueba_IR_UV[i,j,0] = 0
            img_prueba_IR_UV[i,j,1] = 0
            img_prueba_IR_UV[i,j,2] = 1

In [ ]:
visualizar_imagen(img_prueba_IR_UV)

# Animación de canales

Finalmente, vamos a juntar todo lo que hemos construido hasta ahora para hacer una animación de pixeles oscilando en regiones de la imagen donde hay alta intesidad de UV o IR.

In [ ]:
# El número de imágenes del cual estará formada la animación.
num_imagenes = 20

In [ ]:
for n in range(num_imagenes):

    # Hacemos una copia de nuestra imagen RGB, a la cual le vamos
    # a cambiar los colores de los pixeles que estén por arriba
    # de los umbrales UV e IR.
    img_IR_UV = img_RGB_01.copy()

    print(n)

    # Repite lo que hicimos arriba, con las condiciones de
    # UV e IR, pero dentro del for loop que itera sobre el
    # número de imágenes.
    ################################



    #################################
    visualizar_imagen(img_IR_UV)
    pylab.savefig("frame" + str(n) + ".png")


In [ ]:
makemovie(n, "pelicula_UV_IR.gif")

Visualiza el resultado.

In [ ]:
![](pelicula_UV_IR.gif)

**¡Terminamos con este cuaderno! 🎉**

En resumen, lo que hicimos fue procesar información real de una galaxia, la transformamos y añadimos 2 canales de color prácticamente imperceptibles a la vista dentro de información que sí podemos percibir.

---
> Contenido curado por **Rodolfo Ferro**. Contacto: [ferro@cimat.mx](ferro@cimat.mx) <br>
[**Clubes de Ciencia México**](https://clubesdeciencia.mx/), 2025.